---
title: Case Study - Handwritten Digit Recognition with K-Means
subject: Inner Products and Norms
subtitle: 
short_title: Case Study - Handwritten Digit Recognition with K-Means
authors:
  - name: Nikolai Matni
    affiliations:
      - Dept. of Electrical and Systems Engineering
      - University of Pennsylvania
    email: nmatni@seas.upenn.edu
license: CC-BY-4.0
keywords: K-Means, Clustering, MNIST
math:
  '\vv': '\mathbf{#1}'
  '\bm': '\begin{bmatrix}'
  '\em': '\end{bmatrix}'
  '\R': '\mathbb{R}'
---

In this case study, we're going to apply the [k-means algorithm](#k-means-alg) to a dataset consisting of 60000 handwritten digits. 

# The MNIST Dataset

The MNIST dataset, created by Yann Lecun, is a dataset consisting on 60000 handwritten digits, which are represented by $28 \times 28$ pixel black and white images. A few examples of images from the dataset are below:

![25 images from MNIST](../figures/04-mnist_sample.png)


